In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd

from geocube.api.core import make_geocube
from geocube.rasterize import rasterize_image
from functools import partial
from rasterio.enums import MergeAlg

import glob
import os

# Load Data

## Rent controlled and subsidized buildings

In [4]:
aff = pd.read_csv("../data/L1/affordable_housing_with_tree_data_nodupes.csv")

# assign each row a value of one to indicate the num of rent controlled locs (per row) for point density below
aff['value'] = 1

aff.head()


#file_list = glob.glob(os.path.join("../data/L1/geocoded/", "*.csv"))

#file_list
#rent_stabilized_L = [pd.read_csv(file, sep = "\t") for file in file_list]
#rent_stabilized = (pd.concat(rent_stabilized_L)
#                   .query("ZIP!='ZIP'&STATUS1!='MULTIPLE DWELLING B'")
#                  .loc[:,["loc", "lat", "lon"]]
#                   .drop_duplicates("loc")
#                   .dropna(subset = ['lat', 'lon'])
#                 )



,Unnamed: 0,loc,bbl,subsidy_program_full,Value Per Unit,boro_name,latitude,longitude,Max Income Restriction(%AMI),Occupancy Demographic,...,Canopy Cover,chm_min,chm_max,chm_mean,chm_median,cd_min,cd_max,cd_mean,cd_median,value
0,0,80 Rutgers Slip New York City,100248007,Section 202/8,51766.513761,Manhattan,40.710802,-73.990193,50,Seniors; Persons with Disabilities,...,16.04,21.0,291.0,134.764170,146.0,0.044609,239.118332,90.908876,106.167206,1
1,1,15 Bialystoker Place New York City,1003360005,Section 202/8,68058.984375,Manhattan,40.715819,-73.983302,50,Seniors; Persons with Disabilities,...,21.35,21.0,318.0,112.407407,113.0,0.020505,261.449127,76.131788,83.310913,1
2,2,69 West 88 Street New York City,1012020001,Section 202/8,69558.284024,Manhattan,40.787934,-73.970605,50,Seniors; Persons with Disabilities,...,12.55,21.0,347.0,104.253299,98.0,0.027345,284.721985,55.348239,50.131321,1
3,3,234 East 2 Street New York City,1003850053,Section 202/8,48930.000000,Manhattan,40.721706,-73.982095,50,Seniors; Persons with Disabilities,...,17.73,21.0,242.0,94.461009,92.0,0.012957,198.564743,67.197187,64.506371,1
4,4,538 East 6 Street New York City,1004010029,Section 202/8,40050.000000,Manhattan,40.724525,-73.982304,50,Seniors; Persons with Disabilities,...,27.00,21.0,237.0,118.431214,121.0,0.005765,194.421280,73.600179,65.462013,1


In [7]:
gfd = gpd.GeoDataFrame(aff, geometry = gpd.points_from_xy(aff.longitude, aff.latitude), crs=4326)


# Transform to metered projection
# NAD83 (National Spatial Reference System 2011) Universal Transverse Mercator Zone 18 North projection
gfd = gfd.to_crs(6347)

In [10]:
gfd['subsidy_program_full'].value_counts()

Rent Stabilization             27954
421-a Tax Incentive Program     4898
J-51 Tax Incentive              2752
420-c Tax Incentive Program     1936
LIHTC 4%                         538
Federal Public Housing           502
Project-Based Section 8          304
LIHTC 9%                         276
Mitchell-Lama                    134
Section 202/8                    128
Loan Management Set-Aside         66
NYCHA - Mixed Financing            4
Name: subsidy_program_full, dtype: int64

In [8]:
# Create point density (sum of affordable housing per 100m^2 grid cell)
pt_density = make_geocube(vector_data = gfd,
                           measurements = ['value'],
                           resolution = (-100,100), # hectare resolution
                           rasterize_function = partial(rasterize_image, merge_alg = MergeAlg.add),
                           fill = 0) 

In [9]:
# Save rent stabilized point density
pt_density.rio.to_raster('../data/L1/affordablehousing_pointdens_ha.tiff')

## Density of rent stabilized separately

In [13]:
s = gfd.query("subsidy_program_full=='Rent Stabilization'")

# Create point density (sum of affordable housing per 100m^2 grid cell)
pt_density = make_geocube(vector_data = s,
                           measurements = ['value'],
                           resolution = (-100,100), # hectare resolution
                           rasterize_function = partial(rasterize_image, merge_alg = MergeAlg.add),
                           fill = 0) 

# Save rent stabilized point density
pt_density.rio.to_raster('../data/L1/rentstabilized_pointdens_ha.tiff')

## Density of subsidized separately

In [14]:
s = gfd.query("subsidy_program_full!='Rent Stabilization'")

# Create point density (sum of affordable housing per 100m^2 grid cell)
pt_density = make_geocube(vector_data = s,
                           measurements = ['value'],
                           resolution = (-100,100), # hectare resolution
                           rasterize_function = partial(rasterize_image, merge_alg = MergeAlg.add),
                           fill = 0) 

# Save rent stabilized point density
pt_density.rio.to_raster('../data/L1/subsidized_pointdens_ha.tiff')

,Unnamed: 0,loc,bbl,subsidy_program_full,Value Per Unit,boro_name,latitude,longitude,Max Income Restriction(%AMI),Occupancy Demographic,...,chm_min,chm_max,chm_mean,chm_median,cd_min,cd_max,cd_mean,cd_median,value,geometry
0,0,80 Rutgers Slip New York City,100248007,Section 202/8,51766.513761,Manhattan,40.710802,-73.990193,50,Seniors; Persons with Disabilities,...,21.0,291.0,134.764170,146.0,0.044609,239.118332,90.908876,106.167206,1,POINT (585297.142 4507144.438)
1,1,15 Bialystoker Place New York City,1003360005,Section 202/8,68058.984375,Manhattan,40.715819,-73.983302,50,Seniors; Persons with Disabilities,...,21.0,318.0,112.407407,113.0,0.020505,261.449127,76.131788,83.310913,1,POINT (585872.753 4507708.039)
2,2,69 West 88 Street New York City,1012020001,Section 202/8,69558.284024,Manhattan,40.787934,-73.970605,50,Seniors; Persons with Disabilities,...,21.0,347.0,104.253299,98.0,0.027345,284.721985,55.348239,50.131321,1,POINT (586851.300 4515725.907)
3,3,234 East 2 Street New York City,1003850053,Section 202/8,48930.000000,Manhattan,40.721706,-73.982095,50,Seniors; Persons with Disabilities,...,21.0,242.0,94.461009,92.0,0.012957,198.564743,67.197187,64.506371,1,POINT (585967.084 4508362.749)
4,4,538 East 6 Street New York City,1004010029,Section 202/8,40050.000000,Manhattan,40.724525,-73.982304,50,Seniors; Persons with Disabilities,...,21.0,237.0,118.431214,121.0,0.005765,194.421280,73.600179,65.462013,1,POINT (585945.848 4508675.462)
